In [1]:
import os
import sys

import urllib.request
# from zipfile import ZipFile

import pandas as pd
import numpy as np

import random

# mover esto a exploring
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib as mpl

# mpl.rcParams.update( mpl.rcParamsDefault ) 

In [5]:
bicicletas = {
    "recorridos" : {
        "2020" : {
            "url" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2020.zip',
            "zip_file" : "./datasets/recorridos-realizados-2020.zip",
            "df" : pd.DataFrame(),
        },
        "2021" : {
            "url" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2021.zip',   
            "zip_file" : "./datasets/recorridos-realizados-2021.zip",
            "df" : pd.DataFrame(),
        },
    },
    "recorridos_df" : pd.DataFrame(),
    "estaciones" : pd.DataFrame(),
}


In [6]:
"""
Descargar recorridios bicicletas
"""
def descargar_recoridos(
    recorridos,
    equivalencias_encabezados
):
    for r in recorridos:
        if not os.path.exists( recorridos[r]["zip_file"] ):
            print("Descargando", recorridos[r]["url"])
            urllib.request.urlretrieve(
              recorridos[r]["url"],
              recorridos[r]["zip_file"]
            )
        else:
            print("ya descargado", recorridos[r]["zip_file"])  
        recorridos[r]["df"] = pd.read_csv(
            recorridos[r]["zip_file"],
            compression = 'zip',
            encoding = 'utf-8', 
        )
#         print( recorridos[r]["df"].info() )
    # ajustar 2021
    recorridos["2021"]["df"] = recorridos["2021"]["df"]
    recorridos["2021"]["df"] = recorridos["2021"]["df"].rename( columns = equivalencias_encabezados )
    recorridos["2021"]["df"]["id"] = recorridos["2021"]["df"]['id'].astype('Int64')
    print('done')
    return recorridos

equivalencias_encabezados = {
        'ID'                                  : 'id',
        'Duración'                            : 'duracion_recorrido',
        'Estado cerrado'                      : 'estado_cerrado',
        'ID de ciclista'                      : 'id_usuario',
        'Tipo de ciclista'                    : 'cilicsta_tipo',
        'Modelo de bicicleta'                 : 'bicicleta_modelo',   

        'Id de estación de inicio'            : 'id_estacion_origen', 
        'Nombre de estación de inicio'        : 'nombre_estacion_origen',
        'Fecha de inicio'                     : 'fecha_origen_recorrido',     
        'Origen de viaje'                     : 'direccion_estacion_origen',  
        # '':'long_estacion_origen',        
        # '':'lat_estacion_origen',
        'Id de estación de fin de viaje'      : 'id_estacion_destino',     
        'Nombre de estación de fin de viaje'  : 'nombre_estacion_destino',
        'Fecha de fin'                        : 'fecha_destino_recorrido',          
        # '':'direccion_estacion_destino',           
        # '':'long_estacion_destino',       
        # '':'lat_estacion_destino',
        # '':'periodo',  
}
bicicletas["recorridos"] = descargar_recoridos(
    bicicletas["recorridos"],
    equivalencias_encabezados
)

ya descargado ./datasets/recorridos-realizados-2020.zip
ya descargado ./datasets/recorridos-realizados-2021.zip
done


In [7]:
bicicletas["recorridos"]["2021"]["df"].info()
bicicletas["recorridos"]["2020"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328726 entries, 0 to 1328725
Data columns (total 13 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   id                         1328726 non-null  Int64  
 1   estado_cerrado             1328726 non-null  object 
 2   duracion_recorrido         1328726 non-null  int64  
 3   id_estacion_origen         1328726 non-null  int64  
 4   fecha_origen_recorrido     1328726 non-null  object 
 5   nombre_estacion_origen     1328726 non-null  object 
 6   fecha_destino_recorrido    1328726 non-null  object 
 7   id_estacion_destino        1328723 non-null  float64
 8   nombre_estacion_destino    1328723 non-null  object 
 9   id_usuario                 1328726 non-null  int64  
 10  cilicsta_tipo              1328726 non-null  object 
 11  bicicleta_modelo           1328726 non-null  object 
 12  direccion_estacion_origen  1328726 non-null  object 
dtypes: Int64(1),

In [8]:
# concatenar recorridos
orden_filtro = [
    'id',
    'duracion_recorrido',
#     'estado_cerrado',
    'id_usuario',
#     'cilicsta_tipo',
#     'bicicleta_modelo',    
    'id_estacion_origen',
    'nombre_estacion_origen',       
#     'direccion_estacion_origen',
    'fecha_origen_recorrido',       
#     'long_estacion_origen',     
#     'lat_estacion_origen',
    'id_estacion_destino',         
    'nombre_estacion_destino',    
#     'direccion_estacion_destino',
    'fecha_destino_recorrido',      
#     'long_estacion_destino',       
#     'lat_estacion_destino', 
    
#     'periodo',                       
]
# def contcatenar_recorridos(recorridos)
frames = []
for r in bicicletas["recorridos"]:
    frames.append( bicicletas["recorridos"][r]["df"] )

bicicletas["recorridos_df"] = pd.concat( frames )
bicicletas["recorridos_df"] = bicicletas["recorridos_df"][orden_filtro]
# bicicletas["recorridos_df"]["id"] = bicicletas["recorridos_df"]["id"].interpolate( method='linear' )
bicicletas["recorridos_df"]["id"] = bicicletas["recorridos_df"]["id"].fillna(0)
bicicletas["recorridos_df"] = bicicletas["recorridos_df"].sort_values(by=['fecha_origen_recorrido'])
bicicletas["recorridos_df"] 

,id,duracion_recorrido,id_usuario,id_estacion_origen,nombre_estacion_origen,fecha_origen_recorrido,id_estacion_destino,nombre_estacion_destino,fecha_destino_recorrido
593758,0,6563,506009,30,030 - Peña,2020-01-01 00:02:50,30.0,030 - Peña,2020-01-01 01:52:13
687904,0,1150,116847,206,159 - Fitz Roy & Gorriti,2020-01-01 00:04:19,207.0,123 - Armenia,2020-01-01 00:23:29
1026453,0,383,259973,308,251 - Solar de la Abadía,2020-01-01 00:05:23,289.0,255 - BARRANCAS DE BELGRANO,2020-01-01 00:11:46
1876322,0,72,354594,92,092 - Salcedo,2020-01-01 00:05:42,92.0,092 - Salcedo,2020-01-01 00:06:54
1126779,0,769,222181,41,041 - Urquiza y Rondeau,2020-01-01 00:07:30,41.0,041 - Urquiza y Rondeau,2020-01-01 00:20:19
...,...,...,...,...,...,...,...,...,...
4,11204919,591,35926,212,129 - Velasco,2021-06-02 16:09:32,158.0,158 - VILLARROEL,2021-06-02 16:19:23
3,11204920,400,713938,117,117 - HUMBERTO 1°,2021-06-02 16:09:34,175.0,147 - Constitución,2021-06-02 16:16:14
2,11204928,299,690073,232,252 - Tamborini,2021-06-02 16:10:08,386.0,277 - Coghlan,2021-06-02 16:15:07
1,11204938,406,348428,387,278 - Donato Alvarez,2021-06-02 16:11:01,262.0,349 - Plaza Saenz Peña,2021-06-02 16:17:47


In [1]:
# NO_FUNCO!
# ajustar ID 2020

# IDs = bicicletas["recorridos_df"]["id"].dropna().unique()
# # list1 = IDs.tolist()

# id_vacios = np.where( 
#     pd.isna( bicicletas["recorridos_df"]["id"] )
# )
    
# for i in id_vacios[0]:
# #     nuevo_id = random.choice( list( set( range( min(list1) + 1, max(list1))) -set(list1) ) )
# #     number = next(iter(set(range(min(IDs)+1, max(IDs))) - set(IDs)))
# #     m = max(IDs)
# #     number = random.choice([x for x in range(m) if x not in IDs])
#     bicicletas["recorridos_df"].iloc[ i, bicicletas["recorridos_df"].columns.get_loc('id')] = i
# #     list1.append(nuevo_id)
    
# bicicletas["recorridos_df"]


### Procesar fecha

Parsear object datetime a dia, mes, año

In [9]:
# bicis_recorridos_2020_df['fecha_origen_recorrido']
formato = '%Y%m%d %H:%M:%S'
bicicletas["recorridos_df"]['start_datetime'] = pd.to_datetime(
    bicicletas["recorridos_df"]['fecha_origen_recorrido'], 
    format = formato
)
# bicicletas["recorridos_df"]['start_date'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%Y-%m-%d')
bicicletas["recorridos_df"]['anio'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%Y')
bicicletas["recorridos_df"]['mes']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%b') 
bicicletas["recorridos_df"]['ndia'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%j') # pasar a lower 
bicicletas["recorridos_df"]['dia']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%a') # pasar a lower
bicicletas["recorridos_df"]['wdia'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%w')
bicicletas["recorridos_df"]['hora'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%H')
bicicletas["recorridos_df"]['mins'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%M')
bicicletas["recorridos_df"]

,id,duracion_recorrido,id_usuario,id_estacion_origen,nombre_estacion_origen,fecha_origen_recorrido,id_estacion_destino,nombre_estacion_destino,fecha_destino_recorrido,start_datetime,anio,mes,ndia,dia,wdia,hora,mins
593758,0,6563,506009,30,030 - Peña,2020-01-01 00:02:50,30.0,030 - Peña,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,Jan,001,Wed,3,00,02
687904,0,1150,116847,206,159 - Fitz Roy & Gorriti,2020-01-01 00:04:19,207.0,123 - Armenia,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,Jan,001,Wed,3,00,04
1026453,0,383,259973,308,251 - Solar de la Abadía,2020-01-01 00:05:23,289.0,255 - BARRANCAS DE BELGRANO,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,Jan,001,Wed,3,00,05
1876322,0,72,354594,92,092 - Salcedo,2020-01-01 00:05:42,92.0,092 - Salcedo,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,Jan,001,Wed,3,00,05
1126779,0,769,222181,41,041 - Urquiza y Rondeau,2020-01-01 00:07:30,41.0,041 - Urquiza y Rondeau,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,Jan,001,Wed,3,00,07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,11204919,591,35926,212,129 - Velasco,2021-06-02 16:09:32,158.0,158 - VILLARROEL,2021-06-02 16:19:23,2021-06-02 16:09:32,2021,Jun,153,Wed,3,16,09
3,11204920,400,713938,117,117 - HUMBERTO 1°,2021-06-02 16:09:34,175.0,147 - Constitución,2021-06-02 16:16:14,2021-06-02 16:09:34,2021,Jun,153,Wed,3,16,09
2,11204928,299,690073,232,252 - Tamborini,2021-06-02 16:10:08,386.0,277 - Coghlan,2021-06-02 16:15:07,2021-06-02 16:10:08,2021,Jun,153,Wed,3,16,10
1,11204938,406,348428,387,278 - Donato Alvarez,2021-06-02 16:11:01,262.0,349 - Plaza Saenz Peña,2021-06-02 16:17:47,2021-06-02 16:11:01,2021,Jun,153,Wed,3,16,11


## Filtrar columnas que no se usen

- fecha_origen_recorrido
- id_estacion_origen
- fecha_destino_recorrido
- direccion_estacion_origen
- long_estacion_origen
- lat_estacion_origen
- direccion_estacion_destino
- long_estacion_destino
- lat_estacion_destino
- periodo



### Chequear
- Tipo de ciclista? Solo hay categoria member
- Origen de viaje? Code o Key Card o Mobile solo registros con code

In [ ]:
# ya lo vole
modelos_de_bicicleta = bicis_recorridos_2021_df['Modelo de bicicleta'].unique()
print('Modelos de bicicleta: '+ modelos_de_bicicleta.__str__())
tipos_de_ciclista = bicis_recorridos_2021_df['Tipo de ciclista'].unique()
print('Tipos de ciclista: '+ tipos_de_ciclista.__str__())
origen_de_viaje = bicis_recorridos_2021_df['Origen de viaje'].unique()
print('Origenes de viaje: '+ origen_de_viaje.__str__())

In [ ]:
# ya lo vole
columns_to_drop = [
    'fecha_origen_recorrido',
    'id_estacion_origen',
    'fecha_destino_recorrido',
    'direccion_estacion_origen',
    'long_estacion_origen',
    'lat_estacion_origen',
    'direccion_estacion_destino',
    'long_estacion_destino',
    'lat_estacion_destino',
    'periodo',
    'id_estacion_destino',
    'id_usuario',
]

In [ ]:
cleaned = bicis_recorridos_2020_df.drop(columns_to_drop, axis= 1)
cleaned.head()

## separar las columnas nombre de estacion de origen y nombre de estacion de destino 

Tienen la info correcta del id de las estaciones

In [ ]:
aux_list_2d_inicio = pd.DataFrame(
    cleaned['nombre_estacion_origen'].str.split('-').tolist(),
    columns =['Id de estación de incio', 'Nombre de estación de inicio']
)
aux_list_2d_inicio['Nombre de estación de inicio'] = aux_list_2d_inicio['Nombre de estación de inicio'].str.lower()
aux_list_2d_inicio


Aca divido la columna Nombre de estacion de fin de viaje porque contiene dos campos el Id y el nombre de la estacion de inicio de viaje. Hay que validar que no haya registros Nan

In [ ]:
bicis_recorridos_2020_df['nombre_estacion_destino'] = bicis_recorridos_2020_df['nombre_estacion_destino'].replace(np.nan, '0 - null')
bicis_recorridos_2020_df['nombre_estacion_destino'].isna().sum()
aux_list_2d_fin = pd.DataFrame(bicis_recorridos_2020_df['nombre_estacion_destino'].str.split('-').tolist(), columns =['Id de estación destino','Nombre de estación destino'])
aux_list_2d_fin['Nombre de estación destino'] = aux_list_2d_fin['Nombre de estación destino'].str.lower()
aux_list_2d_fin

Voy a borrar las columnas que fueron normalizadas y agregar lo spliteado al dataframe original


In [ ]:
cleaned = cleaned.drop(['nombre_estacion_destino','nombre_estacion_origen'], axis= 1)
test = pd.concat([cleaned,aux_list_2d_fin, aux_list_2d_inicio ], axis= 1)
test.head()

In [ ]:
test.sort_values(by='start datetime')

Export csv
- Download
- Upload to cloud


In [ ]:
compression_opts = dict(
    method='zip',
    archive_name='recorridos_2020-cleaned.csv'
)  
test.to_csv(
    'recorridos_2020-celaned.zip',
    index=False,compression=compression_opts
)

In [ ]:
from google.colab import files
files.download('recorridos_2020-celaned.zip')

In [ ]:
from google.colab import drive
drive.mount('/drive')


In [ ]:
test.to_csv('/drive/My Drive/Datasets-bicing/recorridos-2020-cleaned-2.csv')

##Retrieve latest

Fetch already cleaned csv from drive


In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = '17liAPD1lyj2wT-Hcbh4ed9-Cn4peoyIq'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('recorridos-2020-cleaned-2.csv') 
bicis_recorridos_2020_df = pd.read_csv('recorridos-2020-cleaned-2.csv')
bicis_recorridos_2020_df.head()

Workingday or Weekend

In [ ]:
dias_habiles = [
     'Monday',
     'Tuesday',
     'Wednesday',
     'Thursday',
     'Friday'
]

df_with_workingday = bicis_recorridos_2020_df.assign(
    dia_habil=[1 if dia in dias_habiles else 0 for dia in bicis_recorridos_2020_df['DIA']]
)


In [ ]:
df_with_workingday

### Agrego datos del clima queda por hacer

Fetch from drive

In [ ]:
id = '14BpBfjQ3IrfeK8CajCWrEdfPg7S0eqc8'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('weather-data-ba.csv') 
weather_data_df = pd.read_csv('weather-data-ba.csv')
weather_data_df.head()
weather_data_df = weather_data_df.filter(['DATE','PRCP','TAVG'])
weather_data_df.isnull().sum(axis=0)
weather_data_df['PRCP'] = weather_data_df['PRCP'].replace(np.nan, 0)
weather_data_df.head()

In [ ]:
recorridos_latest_df = pd.merge(
    left = df_with_workingday,
    right = weather_data_df,
    left_on = 'START DATE',
    right_on = 'DATE'
)

In [ ]:
recorridos_latest_df.head()

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
recorridos_latest_df.to_csv(
    '/drive/My Drive/Datasets-bicing/recorridos-2020-cleaned-weather-workingday-3.csv'
)